### Time series Model - ARIMA & Prophet

1. ### Time series Model?
* Time series are a collection of data points collected at constant time intervals. These are analyzed to determine the long term trend so as to forecast the future or perform some other form of analysis.
* Time series data is an important form of structured data in many different fields, such as finance, economics, ecology, neuroscience, and physics. Anything that is observed or measured at many points in time forms a time series.
#### Prophet
* Prophet is a procedure for forecasting time series data based on an additive model where non-linear trends are fit with yearly, weekly, and daily seasonality, plus holiday effects.

#### ARIMA
* ARIMA stands for Auto-Regressive Integrated Moving Averages. The ARIMA forecasting for a stationary time series is nothing but a linear (like a linear regression) equation. The predictors depend on the parameters (p,d,q) of the ARIMA model:
* Number of AR (Auto-Regressive) terms (p): AR terms are just lags of dependent variable. For instance if p is 5, the predictors for x(t) will be x(t-1)….x(t-5).
* Number of MA (Moving Average) terms (q): MA terms are lagged forecast errors in prediction equation. For instance if q is 5, the predictors for x(t) will be e(t-1)….e(t-5) where e(i) is the difference between the moving average at ith instant and actual value.
* Number of Differences (d): These are the number of nonseasonal differences, i.e. in this case we took the first order difference. So either we can pass that variable and put d=0 or pass the original variable and put d=1. Both will generate same results.

####  Process:
* Load the data
* Preprocess the data
* Split the data into train and test
* Build the model
* Make predictions
* Evaluate and visualize results


In [1]:
#Importing necessary libraries
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
#change scientific notation to float
np.set_printoptions(formatter={'float_kind':'{:f}'.format})
#increase the size of sns plots
sns.set(rc={'figure.figsize':(11.7,8.27)})

#ARIMA
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.tsa.api import ExponentialSmoothing, SimpleExpSmoothing, Holt
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn import linear_model

import pmdarima as pm
from pmdarima.model_selection import train_test_split
from prophet import Prophet

import itertools
import warnings
import datetime
from datetime import datetime, timedelta
warnings.filterwarnings('ignore')

c:\Users\didit\Downloads\time-series\time-series\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Importing plotly failed. Interactive plots will not work.


### Problem Formulation
* Solving to forecast the GDP per capita of a country for the next 10 years

In [2]:
data = pd.read_csv('gdp_raw_data.csv', encoding='latin-1')
data.head()

,SP_DYN_TFRT_IN,"Fertility rate, total (births per woman)",AUS,Australia,1970,2.86,Unnamed: 6,Unnamed: 7
0,SP_DYN_TFRT_IN,"Fertility rate, total (births per woman)",AUS,Australia,1971,2.96,NaN,NaN
1,SP_DYN_TFRT_IN,"Fertility rate, total (births per woman)",AUS,Australia,1972,2.74,NaN,NaN
2,SP_DYN_TFRT_IN,"Fertility rate, total (births per woman)",AUS,Australia,1973,2.49,NaN,NaN
3,SP_DYN_TFRT_IN,"Fertility rate, total (births per woman)",AUS,Australia,1974,2.40,NaN,NaN
4,SP_DYN_TFRT_IN,"Fertility rate, total (births per woman)",AUS,Australia,1975,2.15,NaN,NaN


In [3]:
data.shape

(270037, 8)

In [4]:
data = data.rename(columns={'SP_DYN_TFRT_IN':'demo indicator','AUS':'location','Australia':'country','1970':'time','Fertility rate, total (births per woman)':'indicator','2.86':'value','Unnamed: 6':'flag codes','Unnamed: 7':'flags'})
data.columns

Index(['demo indicator', 'indicator', 'location', 'country', 'time', 'value',
       'flag codes', 'flags'],
      dtype='object')

In [5]:
data.isnull().sum()

demo indicator         0
indicator              0
location               0
country                0
time                   0
value               2066
flag codes        267971
flags             267971
dtype: int64

### Data Pre-processing 

In [6]:
data['indicator'][data['value'].isnull()].unique()

array(['Population growth (annual %)',
       'Rural population (% of total population)',
       'GDP deflator (base year varies by country)',
       'GDP growth (annual %)',
       'Poverty headcount ratio at $1.90 a day (PPP) (% of population)',
       'PPP conversion factor, GDP (LCU per international $)',
       'Total debt service (% of GNI)', 'GNI per capita (current LCU)',
       'Official exchange rate (LCU per US$, period average)',
       'GDP per capita (current LCU)',
       'PPP conversion factor, private consumption (LCU per international $)',
       'DEC alternative conversion factor (LCU per US$)'], dtype=object)

In [7]:
#remove  null values
data.dropna(subset=['value'])
#filter out the last 2 columns
data = data.iloc[:, :-2]
data.isnull().sum()

demo indicator       0
indicator            0
location             0
country              0
time                 0
value             2066
dtype: int64

In [8]:
#Investigate all the elements within each feature
for column in data.columns:
    print(column)
    print(data[column].unique())

demo indicator
['SP_DYN_TFRT_IN' 'SP_DYN_LE00_IN' 'SP_DYN_IMRT_IN' 'SP_POP_GROW'
 'SH_DYN_AIDS_ZS' 'SP_RUR_TOTL_ZS' 'NY_GDP_MKTP_CN' 'NY_GDP_MKTP_CD'
 'NY_GDP_DEFL_ZS' 'NY_GDP_MKTP_KD_ZG' 'NY_GDP_PCAP_CD' 'NY_GDP_PCAP_PP_CD'
 'NY_GDP_MKTP_PP_CD' 'XTGOV_IMF' 'NY_GNP_PCAP_CD' 'NY_GNP_PCAP_PP_CD'
 'SI_POV_DDAY' 'PA_NUS_PPP' 'DT_TDS_DECT_GN_ZS' '200101' '200151' '200345'
 '200343' '200144' 'NY_GNP_PCAP_CN' 'NY_GDP_MKTP_KD' 'PA_NUS_PPPC_RF'
 'PA_NUS_FCRF' 'NY_GDP_MKTP_KN' 'PA_NUS_ATLS' 'PA_NUS_PRVT_PP'
 'NY_GNP_MKTP_CN' 'NY_GDP_PCAP_PP_KD' 'NY_GDP_MKTP_PP_KD' 'NY_GDP_PCAP_CN']
indicator
['Fertility rate, total (births per woman)'
 'Life expectancy at birth, total (years)'
 'Mortality rate, infant (per 1,000 live births)'
 'Population growth (annual %)'
 'Prevalence of HIV, total (% of population ages 15-49)'
 'Rural population (% of total population)' 'GDP (current LCU)'
 'GDP (current US$)' 'GDP deflator (base year varies by country)'
 'GDP growth (annual %)' 'GDP per capita (current US$)'

In [9]:
for column in data:
    unique_vals = data[column].unique()
    nr_values = len(unique_vals)
    if nr_values < 36:
        print("The number of values for feature {} is: {} -- {}".format(column, nr_values, unique_vals))
    else:
        print("The number of values for feature {} is: {}".format(column, nr_values))

The number of values for feature demo indicator is: 35 -- ['SP_DYN_TFRT_IN' 'SP_DYN_LE00_IN' 'SP_DYN_IMRT_IN' 'SP_POP_GROW'
 'SH_DYN_AIDS_ZS' 'SP_RUR_TOTL_ZS' 'NY_GDP_MKTP_CN' 'NY_GDP_MKTP_CD'
 'NY_GDP_DEFL_ZS' 'NY_GDP_MKTP_KD_ZG' 'NY_GDP_PCAP_CD' 'NY_GDP_PCAP_PP_CD'
 'NY_GDP_MKTP_PP_CD' 'XTGOV_IMF' 'NY_GNP_PCAP_CD' 'NY_GNP_PCAP_PP_CD'
 'SI_POV_DDAY' 'PA_NUS_PPP' 'DT_TDS_DECT_GN_ZS' '200101' '200151' '200345'
 '200343' '200144' 'NY_GNP_PCAP_CN' 'NY_GDP_MKTP_KD' 'PA_NUS_PPPC_RF'
 'PA_NUS_FCRF' 'NY_GDP_MKTP_KN' 'PA_NUS_ATLS' 'PA_NUS_PRVT_PP'
 'NY_GNP_MKTP_CN' 'NY_GDP_PCAP_PP_KD' 'NY_GDP_MKTP_PP_KD' 'NY_GDP_PCAP_CN']
The number of values for feature indicator is: 35 -- ['Fertility rate, total (births per woman)'
 'Life expectancy at birth, total (years)'
 'Mortality rate, infant (per 1,000 live births)'
 'Population growth (annual %)'
 'Prevalence of HIV, total (% of population ages 15-49)'
 'Rural population (% of total population)' 'GDP (current LCU)'
 'GDP (current US$)' 'GDP deflator 

In [10]:
#filter out the data we need GDP per capita
forecast_data = data[data['indicator'] == 'GDP per capita, PPP (current international $)']

forecast_data = forecast_data[~forecast_data['country'].isin(['Dijibouti','South Sudan','Curabao','Eritrea','Bermuda','Venezuela (Bolivarian Republic of)'])]
forecast_data.head()

,demo indicator,indicator,location,country,time,value
76847,NY_GDP_PCAP_PP_CD,"GDP per capita, PPP (current international $)",AUS,Australia,1990,17333.10
76848,NY_GDP_PCAP_PP_CD,"GDP per capita, PPP (current international $)",AUS,Australia,1991,17793.60
76849,NY_GDP_PCAP_PP_CD,"GDP per capita, PPP (current international $)",AUS,Australia,1992,18189.34
76850,NY_GDP_PCAP_PP_CD,"GDP per capita, PPP (current international $)",AUS,Australia,1993,19130.89
76851,NY_GDP_PCAP_PP_CD,"GDP per capita, PPP (current international $)",AUS,Australia,1994,20063.81


In [11]:
forecast_data = forecast_data[['country','time','value']]
forecast_data.head()

,country,time,value
76847,Australia,1990,17333.10
76848,Australia,1991,17793.60
76849,Australia,1992,18189.34
76850,Australia,1993,19130.89
76851,Australia,1994,20063.81


In [12]:
#change time to date- most time series models requires date structure
forecast_data['time'] = forecast_data['time'].astype('str') + '-01'
forecast_data['time'] = pd.to_datetime(forecast_data['time'])
forecast_data.head()

,country,time,value
76847,Australia,1990-01-01,17333.10
76848,Australia,1991-01-01,17793.60
76849,Australia,1992-01-01,18189.34
76850,Australia,1993-01-01,19130.89
76851,Australia,1994-01-01,20063.81


In [13]:
forecast_data.tail()

,country,time,value
267164,San Marino,2014-01-01,59819.24
267165,San Marino,2015-01-01,60262.54
267166,San Marino,2016-01-01,61806.13
267167,San Marino,2017-01-01,63548.59
267168,Curaçao,2011-01-01,27907.87


### Splitting the raw data

In [14]:
X_train = forecast_data[forecast_data['time'] < '2012-01-01']
X_valid = forecast_data[forecast_data['time'] >= '2011-01-01']

print(X_train.shape, X_valid.shape)

X_train['time'].unique()

(3968, 3) (1324, 3)


<DatetimeArray>
['1990-01-01 00:00:00', '1991-01-01 00:00:00', '1992-01-01 00:00:00',
 '1993-01-01 00:00:00', '1994-01-01 00:00:00', '1995-01-01 00:00:00',
 '1996-01-01 00:00:00', '1997-01-01 00:00:00', '1998-01-01 00:00:00',
 '1999-01-01 00:00:00', '2000-01-01 00:00:00', '2001-01-01 00:00:00',
 '2002-01-01 00:00:00', '2003-01-01 00:00:00', '2004-01-01 00:00:00',
 '2005-01-01 00:00:00', '2006-01-01 00:00:00', '2007-01-01 00:00:00',
 '2008-01-01 00:00:00', '2009-01-01 00:00:00', '2010-01-01 00:00:00',
 '2011-01-01 00:00:00']
Length: 22, dtype: datetime64[ns]

### ARIMA ( Autoregressive Integrated Moving Average ) -1 Country
* ARIMA is a model that can be fitted to time series data in order to better understand or predict future points in the series.
* ARIMA models are denoted with the notation ARIMA(p, d, q). These three parameters account for seasonality, trend, and noise in data.
* ARIMA models are applied in some cases where data show evidence of non-stationarity, where an initial differencing step (corresponding to the 'integrated' part of the model) can be applied one or more times to eliminate the non-stationarity.
* The parameters of the ARIMA model are defined as follows:
    * p: The number of lag observations included in the model, also called the lag order.
    * d: The number of times that the raw observations are differenced, also called the degree of differencing.
    * q: The size of the moving average window, also called the order of moving average.

In [15]:
#filter the train data
au  = X_train[X_train['country'] == 'Australia']
au2 = au[['time','value']]
#filter the valid data
au_v = X_valid[X_valid['country'] == 'Australia']
au_v2 = au_v[['time','value']]
#set the data into index
au2.set_index('time', inplace=True)
au_v2.set_index('time', inplace=True)
#set the index for 6 years
index_7_years = pd.date_range(au2.index[-1], freq='AS', periods=7)
index_7_years 

DatetimeIndex(['2011-01-01', '2012-01-01', '2013-01-01', '2014-01-01',
               '2015-01-01', '2016-01-01', '2017-01-01'],
              dtype='datetime64[ns]', freq='AS-JAN')